<a href="https://colab.research.google.com/github/Reennon/gen-ai-cv-lab-1/blob/main/notebooks/autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autoencoder CIFAR-10 Training and Experimentation

This notebook demonstrates training Autoencoder model on the CIFAR-10 dataset using a modular training pipeline implemented in PyTorch Lightning.


In [ ]:
!git clone https://github.com/Reennon/gen-ai-cv-lab-1.git
%cd gen-ai-cv-lab-1

In [ ]:
!pip install -r requirements.txt

In [4]:
!git pull

Already up to date.


In [5]:
import os
import dotenv
import wandb
import torch

from omegaconf import OmegaConf
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from google.colab import userdata

from src.visualization.base_visualizer import BaseVisualizer
from src.training.trainer import train_model
from src.models.autoencoder import Autoencoder


In [6]:
!echo WANDB_KEY=userdata.get("wandb_key") >> .env

In [7]:
dotenv.load_dotenv()

True

In [8]:
parameters = OmegaConf.load("./params/autoencoder.yml")
wandb.login(key=os.environ["WANDB_KEY"])

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
wandb_project_name = "cifar-10-autoencoder"
device = "gpu"

In [10]:
# Define data transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

# Load datasets
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
val_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)


100%|██████████| 170M/170M [00:03<00:00, 49.1MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [11]:
dataiter = iter(train_loader)
images, labels = next(dataiter)
print(torch.min(images), torch.max(images))

tensor(-1.) tensor(1.)


In [13]:
images.shape

torch.Size([64, 3, 32, 32])

In [12]:
hparams = parameters.hyperparameters

In [14]:
# Edit hparams dict here as experiemnt, wandb will log the difference
hparams["lr"] = 3e-3

dict(hparams)

{'lr': 0.003, 'epochs': 10, 'latent_dim': 64}

In [15]:
# Train the Autoencoder
train_model(Autoencoder, parameters, train_loader, val_loader, wandb_project_name)


TypeError: BaseModel.__init__() missing 1 required positional argument: 'hparams'

In [ ]:
Autoencoder.__dict__

In [ ]:
# Visualize metrics from wandb
from IPython.display import display
wandb_url = wandb.run.url
display(f"Wandb Dashboard: {wandb_url}")


In [ ]:
# Load the model
model = Autoencoder(hparams)
model.load_from_checkpoint('path/to/best_checkpoint.ckpt')
model.eval()

# Visualize original and reconstructed images
visualizer = BaseVisualizer(model, val_loader)
visualizer.visualize_reconstructions()